In [2]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from dotenv import load_dotenv
import os
from tqdm import tqdm # 진행 상황 바를 표시하기 위한 라이브러리

# .env 파일에서 환경 변수 로드 (예: OpenAI API 키)
load_dotenv()

# --- 설정 (Configuration) ---
# OpenAI 임베딩 모델 설정
# "text-embedding-3-large"는 고품질의 임베딩을 제공합니다.
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 텍스트 분할기 설정
# chunk_size: 각 텍스트 청크의 최대 문자 수
# chunk_overlap: 인접한 청크 간의 중복 문자 수 (컨텍스트 유지에 도움)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 벡터 DB 저장 경로 및 대상 디렉토리 설정
# name: 각 DB의 식별 이름
# path: 원본 PDF 파일이 있는 디렉토리 경로 (하위 디렉토리 포함)
# persist_path: 생성될 Chroma 벡터 DB가 저장될 경로
targets = [
    {"name": "DB1", "path": r"C:\skn13\final\DB1", "persist_path": "./vector_db/db1_regulation"},
    {"name": "DB2", "path": r"C:\skn13\final\DB2", "persist_path": "./vector_db/db2_internal"}
]

# 벡터 삽입 시 사용할 배치(Batch) 크기
# 너무 크면 메모리 문제가 발생할 수 있고, 너무 작으면 시간이 오래 걸릴 수 있습니다.
# 50~200 사이에서 조절하는 것이 일반적입니다.
BATCH_SIZE = 100 

# --- 주 처리 로직 (Main Processing Logic) ---
for target in targets:
    print(f"\n--- {target['name']} DB 처리 시작 ---")

    # 1. PDF 문서 로드
    # os.walk를 사용하여 지정된 경로 내의 모든 하위 디렉토리까지 탐색하여 PDF 파일을 찾습니다.
    docs = []
    for root, _, files in os.walk(target["path"]):
        for file in files:
            if file.lower().endswith(".pdf"):
                full_path = os.path.join(root, file)
                try:
                    loader = PyMuPDFLoader(full_path)
                    docs.extend(loader.load())
                except Exception as e:
                    # PDF 로딩 중 오류 발생 시 메시지 출력
                    print(f"⚠️ 문서 로딩 실패: {full_path} → {e}")

    print(f"✔️ 총 로드된 PDF 문서 수: {len(docs)}개")
    if not docs:
        print("💡 로드할 PDF 문서가 없습니다. 다음 DB로 넘어갑니다.")
        continue # 문서가 없으면 현재 DB 처리를 건너뛰고 다음 DB로 이동

    # 2. 로드된 문서를 텍스트 청크로 분할
    chunks = splitter.split_documents(docs)
    # 내용이 비어있는 청크 제거 (간혹 발생할 수 있음)
    chunks = [doc for doc in chunks if doc.page_content.strip()]
    print(f"✔️ 유효한 텍스트 청크 수: {len(chunks)}개")
    
    if not chunks:
        print("💡 유효한 텍스트 청크가 없습니다. 다음 DB로 넘어갑니다.")
        continue # 청크가 없으면 현재 DB 처리를 건너뛰고 다음 DB로 이동

    # 3. Chroma DB 초기화 또는 로드
    # persist_directory 경로에 파일이 존재하고 비어있지 않으면 기존 DB를 로드합니다.
    # 그렇지 않으면 새로운 Chroma DB 인스턴스를 생성합니다.
    # 주의: Chroma.from_documents()는 기존 데이터를 덮어쓸 수 있습니다.
    # 여기서는 Chroma 객체를 먼저 생성하고 add_texts로 청크를 추가하여 기존 데이터에 병합하도록 합니다.
    if os.path.exists(target["persist_path"]) and len(os.listdir(target["persist_path"])) > 0:
        print(f"✨ 기존 {target['name']} DB 로드 중...")
        vectorstore = Chroma(
            collection_name=target["name"], # 컬렉션 이름 지정 (선택 사항이지만 유용)
            embedding_function=embedding_model, # 사용할 임베딩 모델 지정
            persist_directory=target["persist_path"] # DB가 저장된/저장될 디렉토리
        )
    else:
        print(f"✨ 새로운 {target['name']} DB 생성 중...")
        vectorstore = Chroma(
            collection_name=target["name"],
            embedding_function=embedding_model,
            persist_directory=target["persist_path"]
        )
        
    # 4. 분할된 청크를 Chroma DB에 배치(Batch)로 삽입
    print(f"🚀 {target['name']} DB에 청크를 삽입합니다 (총 {len(chunks)}개)...")
    # tqdm을 사용하여 청크 삽입 진행 상황을 시각적으로 표시합니다.
    for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc=f"{target['name']} 청크 삽입"):
        batch = chunks[i:i + BATCH_SIZE] # 현재 배치에 해당하는 청크를 추출
        texts = [doc.page_content for doc in batch] # 청크의 텍스트 내용만 추출
        metadatas = [doc.metadata for doc in batch] # 청크의 메타데이터만 추출

        try:
            # Chroma DB에 텍스트와 메타데이터를 추가
            vectorstore.add_texts(texts=texts, metadatas=metadatas)
        except Exception as e:
            # 벡터 삽입 중 오류 발생 시 메시지 출력
            print(f"❌ 벡터 삽입 실패 (배치 {i // BATCH_SIZE}): {e}")

    print(f"✅ {target['name']} 벡터 DB 생성/업데이트 완료 → {target['persist_path']}")

print("\n--- 모든 벡터 DB 처리 완료 ---")


--- DB1 DB 처리 시작 ---
✔️ 총 로드된 PDF 문서 수: 2172개
✔️ 유효한 텍스트 청크 수: 6067개
✨ 기존 DB1 DB 로드 중...
🚀 DB1 DB에 청크를 삽입합니다 (총 6067개)...


DB1 청크 삽입: 100%|██████████| 61/61 [03:11<00:00,  3.13s/it]


✅ DB1 벡터 DB 생성/업데이트 완료 → ./vector_db/db1_regulation

--- DB2 DB 처리 시작 ---
MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

✔️ 총 로드된 PDF 문서 수: 4794개
✔️ 유효한 텍스트 청크 수: 9378개
✨ 기존 DB2 DB 로드 중...
🚀 DB2 DB에 청크를 삽입합니다 (총 9378개)...


DB2 청크 삽입: 100%|██████████| 94/94 [05:00<00:00,  3.20s/it]

✅ DB2 벡터 DB 생성/업데이트 완료 → ./vector_db/db2_internal

--- 모든 벡터 DB 처리 완료 ---


In [1]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from tqdm import tqdm

# 환경 변수 로드
load_dotenv()

# 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# 텍스트 분할기 설정
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)

# 대상 DB 목록
targets = [
    {"name": "db1_regulation", "path": r"C:\skn13\final\DB1", "persist_path": "./qdrant_db/db1_regulation"},
    {"name": "db2_internal", "path": r"C:\skn13\final\DB2", "persist_path": "./qdrant_db/db2_internal"},
]

BATCH_SIZE = 100

for target in targets:
    print(f"\n--- {target['name']} DB 처리 시작 ---")

    # 문서 로드
    docs = []
    for root, _, files in os.walk(target["path"]):
        for file in files:
            if file.lower().endswith(".pdf"):
                full_path = os.path.join(root, file)
                try:
                    loader = PyMuPDFLoader(full_path)
                    docs.extend(loader.load())
                except Exception as e:
                    print(f"문서 로딩 실패: {full_path} → {e}")

    print(f"총 로드된 문서 수: {len(docs)}개")
    if not docs:
        continue

    # 텍스트 분할
    chunks = splitter.split_documents(docs)
    chunks = [doc for doc in chunks if doc.page_content.strip()]
    print(f"유효 텍스트 청크 수: {len(chunks)}개")
    if not chunks:
        continue

    # Qdrant 로컬 인스턴스 생성 (주의: 커널 재시작 필요할 수 있음)
    try:
        client = QdrantClient(path=target["persist_path"])
    except RuntimeError as e:
        print(f"Qdrant 클라이언트 초기화 실패: {e}")
        continue

    # 벡터 크기 확인 후 컬렉션 생성
    vector_dim = embedding_model.embed_query("테스트 문장입니다.").__len__()

    try:
        client.recreate_collection(
            collection_name=target["name"],
            vectors_config=VectorParams(size=vector_dim, distance=Distance.COSINE),
        )
    except Exception as e:
        print(f"Qdrant 컬렉션 생성 오류: {e}")
        continue

    # LangChain Qdrant 래퍼로 초기화 (주의: embeddings 파라미터 사용)
    vectorstore = Qdrant(
        client=client,
        collection_name=target["name"],
        embeddings=embedding_model,
    )

    print(f"{target['name']}에 청크 삽입 중...")
    for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc=f"{target['name']} 청크 삽입"):
        batch = chunks[i:i + BATCH_SIZE]
        texts = [doc.page_content for doc in batch]
        metadatas = [doc.metadata for doc in batch]
        try:
            vectorstore.add_texts(texts=texts, metadatas=metadatas)
        except Exception as e:
            print(f"벡터 삽입 실패 (배치 {i // BATCH_SIZE}): {e}")

    print(f"{target['name']} 벡터 DB 생성 완료 → {target['persist_path']}")

print("\n--- 모든 Qdrant 벡터 DB 처리 완료 ---")



--- db1_regulation DB 처리 시작 ---
총 로드된 문서 수: 2172개
유효 텍스트 청크 수: 6067개


C:\Users\Playdata\AppData\Local\Temp\ipykernel_13412\1963693632.py:65: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
C:\Users\Playdata\AppData\Local\Temp\ipykernel_13412\1963693632.py:74: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use :class:`~QdrantVectorStore` instead.
  vectorstore = Qdrant(


db1_regulation에 청크 삽입 중...


db1_regulation 청크 삽입: 100%|██████████| 61/61 [04:09<00:00,  4.08s/it]


db1_regulation 벡터 DB 생성 완료 → ./qdrant_db/db1_regulation

--- db2_internal DB 처리 시작 ---
MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

총 로드된 문서 수: 4794개
유효 텍스트 청크 수: 9378개
db2_internal에 청크 삽입 중...


db2_internal 청크 삽입: 100%|██████████| 94/94 [06:18<00:00,  4.03s/it]

db2_internal 벡터 DB 생성 완료 → ./qdrant_db/db2_internal

--- 모든 Qdrant 벡터 DB 처리 완료 ---


In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import Qdrant
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance
from dotenv import load_dotenv
from tqdm import tqdm
import os

load_dotenv()

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
BATCH_SIZE = 100

targets = [
    {"name": "db1_regulation", "path": r"C:\skn13\final\DB1", "persist_path": "./qdrant_db/db1_regulation"},
    {"name": "db2_internal", "path": r"C:\skn13\final\DB2", "persist_path": "./qdrant_db/db2_internal"},
]

for target in targets:
    print(f"\n--- {target['name']} DB 처리 시작 ---")

    docs = []
    for root, _, files in os.walk(target["path"]):
        for file in files:
            if file.lower().endswith(".pdf"):
                try:
                    loader = PyMuPDFLoader(os.path.join(root, file))
                    docs.extend(loader.load())
                except Exception as e:
                    print(f"문서 로딩 실패: {file} → {e}")

    print(f"총 로드된 문서 수: {len(docs)}개")
    chunks = splitter.split_documents(docs)
    chunks = [doc for doc in chunks if doc.page_content.strip()]
    print(f"유효 텍스트 청크 수: {len(chunks)}개")
    if not chunks:
        continue

    client = QdrantClient(path=target["persist_path"])
    collection_name = target["name"]

    dim = len(embedding_model.embed_query("test"))
    if client.collection_exists(collection_name):
        client.delete_collection(collection_name)
    client.create_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=dim, distance=Distance.COSINE)
    )

    vectorstore = Qdrant(
        client=client,
        collection_name=collection_name,
        embeddings=embedding_model
    )

    print(f"{collection_name}에 청크 삽입 중...")
    for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc=f"{collection_name} 청크 삽입"):
        batch = chunks[i:i + BATCH_SIZE]
        texts = [doc.page_content for doc in batch]
        metadatas = [doc.metadata for doc in batch]
        try:
            vectorstore.add_texts(texts=texts, metadatas=metadatas)
        except Exception as e:
            print(f"벡터 삽입 실패 (배치 {i // BATCH_SIZE}): {e}")

    print(f"{collection_name} 벡터 DB 생성 완료 → {target['persist_path']}")

print("\n--- 모든 Qdrant 벡터 DB 처리 완료 ---")



--- db1_regulation DB 처리 시작 ---
총 로드된 문서 수: 2172개
유효 텍스트 청크 수: 6067개


C:\Users\Playdata\AppData\Local\Temp\ipykernel_22248\3811282902.py:53: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.1.2 and will be removed in 0.5.0. Use :class:`~QdrantVectorStore` instead.
  vectorstore = Qdrant(


db1_regulation에 청크 삽입 중...


db1_regulation 청크 삽입: 100%|██████████| 61/61 [04:49<00:00,  4.75s/it]


db1_regulation 벡터 DB 생성 완료 → ./qdrant_db/db1_regulation

--- db2_internal DB 처리 시작 ---
MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict

총 로드된 문서 수: 4794개
유효 텍스트 청크 수: 9378개
db2_internal에 청크 삽입 중...


db2_internal 청크 삽입: 100%|██████████| 94/94 [07:34<00:00,  4.83s/it]

db2_internal 벡터 DB 생성 완료 → ./qdrant_db/db2_internal

--- 모든 Qdrant 벡터 DB 처리 완료 ---


In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(path="./qdrant_db/db2_internal")  # 경로는 실제 DB 위치
print(client.get_collections())


collections=[CollectionDescription(name='DB2'), CollectionDescription(name='db2_internal')]


In [4]:
collection_name = "db2_internal"
info = client.get_collection(collection_name=collection_name)
print(f"총 벡터 수: {info.points_count}")


총 벡터 수: 18756


In [7]:
from langchain_openai import OpenAIEmbeddings
from langchain_qdrant import Qdrant
from dotenv import load_dotenv
load_dotenv()

embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore = Qdrant(
    client=client,
    collection_name="db2_internal",
    embeddings=embedding_model
)

query = "사내 정책에 따라 외부 문서 공유는 어떻게 해야 하나요?"
results = vectorstore.similarity_search(query, k=3)

for i, r in enumerate(results):
    print(f"\n[{i+1}] {r.page_content[:300]}...")  # 앞부분만 출력



[1] 45
- 45 -
이트에 저장하거나 참여인력 개인이 보관할 수 없으며, KOBACO의 파일서버에 
저장하거나 사업 담당자가 지정한 PC에 저장 및 관리하여야함(파일 저장 서버는 
인터넷 연결 금지)
❍ 비공개 자료는 매일 퇴근 시 반납하고, 일반문서의 경우 시건 장치가 된 보관함 등 
안전한 장소에 보관함
❍ 사업 수행으로 생산되는 산출물 및 기록은 비인가자에게 제공ㆍ대여ㆍ열람을 금
지함
❍ 수행사업 관련 자료는 개인 메일함에 저장을 금지하고 전자우편을 이용해 자료
전송이 필요한 경우에는 자체 전자우편을 이용하고(상용메일 사용 금지...

[2] 45
- 45 -
이트에 저장하거나 참여인력 개인이 보관할 수 없으며, KOBACO의 파일서버에 
저장하거나 사업 담당자가 지정한 PC에 저장 및 관리하여야함(파일 저장 서버는 
인터넷 연결 금지)
❍ 비공개 자료는 매일 퇴근 시 반납하고, 일반문서의 경우 시건 장치가 된 보관함 등 
안전한 장소에 보관함
❍ 사업 수행으로 생산되는 산출물 및 기록은 비인가자에게 제공ㆍ대여ㆍ열람을 금
지함
❍ 수행사업 관련 자료는 개인 메일함에 저장을 금지하고 전자우편을 이용해 자료
전송이 필요한 경우에는 자체 전자우편을 이용하고(상용메일 사용 금지...

[3] 45
- 45 -
이트에 저장하거나 참여인력 개인이 보관할 수 없으며, KOBACO의 파일서버에 
저장하거나 사업 담당자가 지정한 PC에 저장 및 관리하여야함(파일 저장 서버는 
인터넷 연결 금지)
❍ 비공개 자료는 매일 퇴근 시 반납하고, 일반문서의 경우 시건 장치가 된 보관함 등 
안전한 장소에 보관함
❍ 사업 수행으로 생산되는 산출물 및 기록은 비인가자에게 제공ㆍ대여ㆍ열람을 금
지함
❍ 수행사업 관련 자료는 개인 메일함에 저장을 금지하고 전자우편을 이용해 자료
전송이 필요한 경우에는 자체 전자우편을 이용하고(상용메일 사용 금지...


In [8]:
%pip install langchain langchain-openai langchain-unstructured unstructured opensearch-py

  Using cached langchain_unstructured-0.1.6-py3-none-any.whl.metadata (3.3 kB)
  Using cached unstructured-0.18.11-py3-none-any.whl.metadata (24 kB)
  Using cached onnxruntime-1.19.2-cp312-cp312-win_amd64.whl.metadata (4.7 kB)
Using cached langchain_unstructured-0.1.6-py3-none-any.whl (7.0 kB)
Using cached onnxruntime-1.19.2-cp312-cp312-win_amd64.whl (11.1 MB)
Using cached unstructured-0.18.11-py3-none-any.whl (1.8 MB)

   -------- ------------------------------- 1/5 [opensearch-py]
   -------- ------------------------------- 1/5 [opensearch-py]
   -------- ------------------------------- 1/5 [opensearch-py]
   -------- ------------------------------- 1/5 [opensearch-py]
  Attempting uninstall: onnxruntime
   -------- ------------------------------- 1/5 [opensearch-py]
    Found existing installation: onnxruntime 1.22.0
   -------- ------------------------------- 1/5 [opensearch-py]
    Uninstalling onnxruntime-1.22.0:
   -------- ------------------------------- 1/5 [opensearch-py]
   

In [9]:
%pip install opensearch-py langchain

Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install opensearch-py langchain langchain-openai langchain-unstructured unstructured tqdm

Note: you may need to restart the kernel to use updated packages.


In [11]:
import os
from langchain_community.document_loaders import PyMuPDFLoader, UnstructuredPDFLoader
from langchain_unstructured import UnstructuredLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import OpenSearchVectorSearch
from opensearchpy import OpenSearch
from tqdm import tqdm
import time

# 설정
directory_path = r"C:\skn13\final\DB1"
index_name = "db1_opensearch"
BATCH_SIZE = 100
CHUNK_SIZE = 500
CHUNK_OVERLAP = 100

# OpenAI 임베딩 모델
embedding_model = OpenAIEmbeddings(model="text-embedding-3-large")

# OpenSearch 클라이언트 설정
opensearch_client = OpenSearch(
    hosts=[{"host": "localhost", "port": 9200}],
    http_auth=("admin", "admin"),
    use_ssl=False,
    verify_certs=False
)

# 문서 로딩 함수
def load_documents(path):
    documents = []
    for fname in os.listdir(path):
        fpath = os.path.join(path, fname)
        try:
            if fname.endswith(".pdf"):
                loader = UnstructuredLoader(fpath)
            elif fname.endswith(".txt"):
                from langchain_community.document_loaders import TextLoader
                loader = TextLoader(fpath, encoding="utf-8")
            elif fname.endswith(".docx"):
                from langchain_community.document_loaders import Docx2txtLoader
                loader = Docx2txtLoader(fpath)
            else:
                print(f"지원하지 않는 확장자: {fname}")
                continue
            documents.extend(loader.load())
        except Exception as e:
            print(f"문서 로드 실패: {fname} - {e}")
    return documents

# 문서 로드
docs = load_documents(directory_path)
print(f"총 문서 수: {len(docs)}")

# 청크 처리
splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP
)
chunks = splitter.split_documents(docs)
print(f"총 청크 수: {len(chunks)}")

# 인덱스 존재 시 삭제
if opensearch_client.indices.exists(index=index_name):
    opensearch_client.indices.delete(index=index_name)
    print(f"기존 인덱스 {index_name} 삭제됨")

# 벡터 저장소 연결
vectorstore = OpenSearchVectorSearch(
    index_name=index_name,
    embedding_function=embedding_model.embed_query,
    opensearch_url="http://localhost:9200",
    http_auth=("admin", "admin")
)

# 벡터 삽입
for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc="벡터 업로드 중"):
    batch = chunks[i:i+BATCH_SIZE]
    try:
        vectorstore.add_documents(batch)
    except Exception as e:
        print(f"배치 {i // BATCH_SIZE} 삽입 실패: {e}")
        time.sleep(1)

print(f"✅ OpenSearch 인덱스 '{index_name}' 벡터 저장 완료")


지원하지 않는 확장자: 관계법령
지원하지 않는 확장자: 법령
지원하지 않는 확장자: 사내규정
총 문서 수: 0
총 청크 수: 0


ConnectionError: ConnectionError(<urllib3.connection.HTTPConnection object at 0x00000110817B4FB0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다) caused by: NewConnectionError(<urllib3.connection.HTTPConnection object at 0x00000110817B4FB0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다)

In [2]:
import os
import base64
from pathlib import Path
from tqdm import tqdm
import fitz  # pymupdf
import pandas as pd

# PDF가 들어있는 루트 디렉토리 경로 설정
pdf_root = Path("C:/skn13/final/DB2")  # 필요에 따라 수정

# 결과 저장용 딕셔너리
markdown_by_file = {}

# 마크다운 변환 함수
def convert_pdf_to_markdown(pdf_path, image_output_dir):
    doc = fitz.open(pdf_path)
    markdown = []
    image_count = 0
    table_count = 0  # 이건 추후 표 처리 라이브러리와 연동 시 사용 가능

    for page_num, page in enumerate(doc):
        markdown.append(f"# Page {page_num + 1}\n")

        # 텍스트 추출
        text = page.get_text("text")
        markdown.append(text.strip())

        # 이미지 추출
        image_list = page.get_images(full=True)
        for img_index, img in enumerate(image_list):
            xref = img[0]
            pix = fitz.Pixmap(doc, xref)
            image_bytes = pix.tobytes("png")
            image_count += 1

            # base64 인코딩
            b64 = base64.b64encode(image_bytes).decode("ascii")
            md_img = f"![image_page{page_num+1}_{img_index}](data:image/png;base64,{b64})"
            markdown.append(md_img)

            # 이미지에 대한 설명 프롬프트 삽입 (멀티모달 처리용)
            markdown.append(f"\n*이 이미지는 멀티모달로 설명 요망: Page {page_num + 1}, Image {img_index}*\n")

    return "\n\n".join(markdown), image_count, table_count

# PDF 일괄 처리
summary = []
pdf_files = list(pdf_root.rglob("*.pdf"))

for pdf in tqdm(pdf_files, desc="PDF → Markdown 변환 중"):
    try:
        md, img_count, tbl_count = convert_pdf_to_markdown(pdf, pdf_root)
        markdown_by_file[str(pdf.relative_to(pdf_root))] = md
        summary.append({
            "file": str(pdf.relative_to(pdf_root)),
            "pages": fitz.open(pdf).page_count,
            "images": img_count,
            "tables": tbl_count
        })
    except Exception as e:
        markdown_by_file[str(pdf.relative_to(pdf_root))] = f"[ERROR] {e}"
        summary.append({
            "file": str(pdf.relative_to(pdf_root)),
            "pages": 0,
            "images": 0,
            "tables": 0
        })

# 요약표 출력
summary_df = pd.DataFrame(summary)
print(summary_df)


PDF → Markdown 변환 중:  45%|████▌     | 114/251 [01:07<00:08, 16.81it/s]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



PDF → Markdown 변환 중: 100%|██████████| 251/251 [01:19<00:00,  3.14it/s]

                                                file  pages  images  tables
0    데이터포털\2023년 고정형TV VOD 시청행태보고서(인쇄의뢰용)_수정0304.pdf    117     112       0
1                데이터포털\2023년 고정형TV 실시간 시청점유율 보고서.pdf     97     127       0
2      데이터포털\2023년 소비지출 트렌드 조사 결과보고서 (4차_라이프스타일).pdf     28      86       0
3              데이터포털\2023년 스마트폰·PC 시청행태 조사 결과보고서.pdf     96     121       0
4                  데이터포털\2023년 시청점유율 기초조사 공개용보고서.pdf    528    1058       0
..                                               ...    ...     ...     ...
246                내부문서\투자\코바코파트너스 회계 감사보고서 2024.pdf      0       0       0
247                 내부문서\투자\한국방송광고진흥공사_경영부담+비용추계.pdf      1       0       0
248                     내부문서\투자\한국방송광고진흥공사_출연+현황.pdf      1       0       0
249                내부문서\투자\한국방송광고진흥공사_투자+및+출자+현황.pdf      2       0       0
250                    내부문서\투자\한국방송광고진흥공사_투자집행내역.pdf      1       0       0

[251 rows x 4 columns]


In [1]:
%pip install git+https://github.com/docling-project/docling.git


  Cloning https://github.com/docling-project/docling.git to c:\users\playdata\appdata\local\temp\pip-req-build-9nldj7gn
  Resolved https://github.com/docling-project/docling.git to commit c5f49dc2db43d38997196b7c49f90ea3b00c07ca
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached easyocr-1.7.2-py3-none-any.whl.metadata (10 kB)
  Using cached rtree-1.4.0-py3-none-win_amd64.whl.metadata (2.1 kB)
  Using cached python_pptx-1.0.2-py3-none-any.whl.metadata (2.5 kB)
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached lxml-5.4.0-cp312-cp312-win_amd64.whl.metadata (3.6 kB)
  Using cached pylatexenc-2.10-py3-none-any.whl
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata 

  Running command git clone --filter=blob:none --quiet https://github.com/docling-project/docling.git 'C:\Users\Playdata\AppData\Local\Temp\pip-req-build-9nldj7gn'
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
effdet 0.4.1 requires pycocotools>=2.0.2, which is not installed.
matplotlib 3.10.3 requires pyparsing>=2.3.1, which is not installed.


In [2]:
%pip install docling


Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import json
import subprocess
from pathlib import Path
from tqdm import tqdm
import fitz  # PyMuPDF

# --- 설정 ---
pdf_root = Path("C:/skn13/final/DB2")         # PDF 디렉토리
output_dir = pdf_root / "_markdown"           # Markdown 출력 디렉토리
image_dir = pdf_root / "_images"              # 이미지 저장 디렉토리

output_dir.mkdir(exist_ok=True)
image_dir.mkdir(exist_ok=True)

# --- 함수 정의 ---
def run_docling_extract(pdf_path: Path) -> dict:
    """docling CLI로 텍스트/표 추출"""
    import tempfile

    with tempfile.NamedTemporaryFile(suffix=".json", delete=False) as tmp_out:
        output_path = tmp_out.name

    subprocess.run([
        "docling", "extract",
        str(pdf_path),
        "--output", output_path
    ], check=True)

    with open(output_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    return data


def extract_images_as_markdown(pdf_path: Path, image_dir: Path) -> list:
    """이미지 추출 후 Markdown 형식으로 반환"""
    doc = fitz.open(pdf_path)
    md_images = []

    for page_index in range(len(doc)):
        for img_index, img in enumerate(doc.get_page_images(page_index)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            img_bytes = base_image["image"]
            ext = base_image["ext"]
            img_filename = f"{pdf_path.stem}_p{page_index+1}_{img_index+1}.{ext}"
            img_path = image_dir / img_filename

            with open(img_path, "wb") as f:
                f.write(img_bytes)

            rel_path = img_path.relative_to(pdf_root)
            md_images.append(f"![이미지 설명]({rel_path.as_posix()})")

    return md_images


# --- 전체 PDF 처리 ---
markdown_by_file = {}
pdfs = list(pdf_root.rglob("*.pdf"))
print(f"총 PDF 수: {len(pdfs)}")

for pdf in tqdm(pdfs, desc="PDF → Markdown 변환 중"):
    try:
        # 텍스트/표 추출
        result = run_docling_extract(pdf)
        sections = result.get("sections", [])
        texts = [s.get("text", "") for s in sections if s.get("type") == "text"]
        tables = [s.get("markdown", "") for s in sections if s.get("type") == "table"]

        # 이미지 추출
        image_markdowns = extract_images_as_markdown(pdf, image_dir)

        # 마크다운 통합
        combined = texts + tables + image_markdowns
        markdown_by_file[str(pdf.relative_to(pdf_root))] = combined

    except Exception as e:
        print(f"{pdf} 처리 중 오류: {e}")
        markdown_by_file[str(pdf.relative_to(pdf_root))] = [f"ERROR: {str(e)}"]

# --- 저장 ---
for filename, chunks in markdown_by_file.items():
    save_path = output_dir / (Path(filename).stem + ".md")
    with open(save_path, "w", encoding="utf-8") as f:
        f.write("\n\n".join(chunks))

print("✅ 모든 PDF를 Markdown 형식으로 저장 완료.")


총 PDF 수: 251


PDF → Markdown 변환 중:   0%|          | 1/251 [00:02<09:08,  2.19s/it]

C:\skn13\final\DB2\데이터포털\2023년 고정형TV VOD 시청행태보고서(인쇄의뢰용)_수정0304.pdf 처리 중 오류: Command '['docling', 'extract', 'C:\\skn13\\final\\DB2\\데이터포털\\2023년 고정형TV VOD 시청행태보고서(인쇄의뢰용)_수정0304.pdf', '--output', 'C:\\Users\\Playdata\\AppData\\Local\\Temp\\tmpdzewba59.json']' returned non-zero exit status 1.


PDF → Markdown 변환 중:   1%|          | 2/251 [00:04<09:07,  2.20s/it]

C:\skn13\final\DB2\데이터포털\2023년 고정형TV 실시간 시청점유율 보고서.pdf 처리 중 오류: Command '['docling', 'extract', 'C:\\skn13\\final\\DB2\\데이터포털\\2023년 고정형TV 실시간 시청점유율 보고서.pdf', '--output', 'C:\\Users\\Playdata\\AppData\\Local\\Temp\\tmpjzfs7ehn.json']' returned non-zero exit status 1.


PDF → Markdown 변환 중:   1%|          | 3/251 [00:06<09:08,  2.21s/it]

C:\skn13\final\DB2\데이터포털\2023년 소비지출 트렌드 조사 결과보고서 (4차_라이프스타일).pdf 처리 중 오류: Command '['docling', 'extract', 'C:\\skn13\\final\\DB2\\데이터포털\\2023년 소비지출 트렌드 조사 결과보고서 (4차_라이프스타일).pdf', '--output', 'C:\\Users\\Playdata\\AppData\\Local\\Temp\\tmpar9rjuno.json']' returned non-zero exit status 1.


PDF → Markdown 변환 중:   1%|          | 3/251 [00:08<12:17,  2.97s/it]


KeyboardInterrupt: 

In [2]:
%pip install PyMuPDF pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install "camelot-py[cv]"

Note: you may need to restart the kernel to use updated packages.


In [1]:
import fitz  # PyMuPDF
import camelot
import pandas as pd
from pathlib import Path
import os
from tqdm import tqdm # 진행상황 표시를 위한 라이브러리

def process_pdf_to_markdown(pdf_path: Path, output_dir: Path, image_dir: Path):
    """
    하나의 PDF 파일을 분석하여 텍스트, 표, 이미지를 추출하고
    설명을 포함한 마크다운 파일로 변환합니다.
    """
    pdf_document = fitz.open(pdf_path)
    markdown_content = []

    # 문서 제목 추가
    markdown_content.append(f"# {pdf_path.stem}\n")

    # 각 페이지 순회
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        page_content_md = [f"\n---\n\n## 페이지 {page_num + 1}\n"]

        # 1. 텍스트 추출
        text = page.get_text("text")
        if text.strip():
            page_content_md.append("### 텍스트 내용\n")
            page_content_md.append(text)

        # 2. 표(Table) 추출 및 변환
        try:
            tables = camelot.read_pdf(str(pdf_path), pages=str(page_num + 1), flavor='lattice')
            if tables.n > 0:
                page_content_md.append("\n### 추출된 표\n")
                for i, table in enumerate(tables):
                    description = f"* **설명**: 페이지 {page_num + 1}에서 {i+1}번째로 발견된 표입니다."
                    page_content_md.append(description)
                    markdown_table = table.df.to_markdown(index=False)
                    page_content_md.append(markdown_table + "\n")
        except Exception:
            # 표 추출에 실패하더라도 다음 단계로 넘어가도록 함
            pass

        # 3. 이미지 추출 및 변환
        images = page.get_images(full=True)
        if images:
            page_content_md.append("\n### 추출된 이미지\n")
            for img_index, img in enumerate(images):
                xref = img[0]
                base_image = pdf_document.extract_image(xref)
                image_bytes = base_image["image"]
                image_ext = base_image["ext"]

                image_filename = f"{pdf_path.stem}_p{page_num + 1}_{img_index + 1}.{image_ext}"
                image_save_path = image_dir / image_filename
                
                with open(image_save_path, "wb") as f:
                    f.write(image_bytes)

                img_description = f"![페이지 {page_num + 1}의 {img_index + 1}번째 이미지]({image_save_path.as_posix()})"
                explanation = f"* **설명**: 페이지 {page_num + 1}에서 추출된 이미지입니다."
                
                page_content_md.append(img_description)
                page_content_md.append(explanation)

        markdown_content.extend(page_content_md)

    # 최종 마크다운 파일 저장
    # 파일 이름에 포함될 수 없는 문자를 제거하거나 다른 문자로 치환
    safe_filename = "".join(c for c in pdf_path.stem if c.isalnum() or c in (' ', '_')).rstrip()
    output_md_path = output_dir / (safe_filename + ".md")
    
    with open(output_md_path, "w", encoding="utf-8") as f:
        f.write("\n".join(markdown_content))

# --- 실행 부분 ---
if __name__ == "__main__":
    # 1. ★★★ PDF 파일이 있는 루트 폴더 경로 설정 ★★★
    pdf_root = Path("C:/skn13/final/DB2")

    # 2. 출력 폴더 설정
    output_directory = pdf_root / "_markdown_output"
    image_directory = output_directory / "_images"

    # 3. 출력 폴더 생성 (없으면)
    output_directory.mkdir(exist_ok=True)
    image_directory.mkdir(exist_ok=True)

    # 4. 지정된 폴더에서 모든 PDF 파일 찾기 (하위 폴더 포함)
    pdf_files = list(pdf_root.rglob("*.pdf"))

    if not pdf_files:
        print(f"오류: '{pdf_root}' 폴더에 PDF 파일이 없습니다.")
    else:
        print(f"총 {len(pdf_files)}개의 PDF 파일을 찾았습니다. 변환을 시작합니다...")
        
        # 5. 찾은 PDF 파일을 하나씩 순회하며 변환 함수 호출
        for pdf_file in tqdm(pdf_files, desc="PDF 변환 진행 중"):
            try:
                process_pdf_to_markdown(pdf_file, output_directory, image_directory)
            except Exception as e:
                print(f"오류 발생: '{pdf_file.name}' 파일 처리 중 문제가 발생했습니다. 원인: {e}")
        
        print(f"\n✅ 모든 작업이 완료되었습니다. 결과는 '{output_directory}' 폴더를 확인하세요.")

총 251개의 PDF 파일을 찾았습니다. 변환을 시작합니다...


PDF 변환 진행 중:   1%|          | 2/251 [02:28<5:07:20, 74.06s/it]c:\Users\Playdata\miniconda3\envs\lang_env\Lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-1 is image-based, camelot only works on text-based pages.
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\lang_env\Lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-2 is image-based, camelot only works on text-based pages.
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\lang_env\Lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-3 is image-based, camelot only works on text-based pages.
  warnings.warn(
c:\Users\Playdata\miniconda3\envs\lang_env\Lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-4 is image-based, camelot only works on text-based pages.
  warnings.warn(
Cannot set gray non-stroke color because /'P31' is an invalid float value
c:\Users\Playdata\miniconda3\envs\lang_env\Lib\site-packages\camelot\parsers\base.py:124: UserWarning: page-6 is image-based, camelo

KeyboardInterrupt: 

In [7]:
import fitz  # PyMuPDF
from pathlib import Path
from tqdm import tqdm
import shutil # 파일 복사를 위해 shutil 라이브러리 추가

# --- 설정 ---
TEXT_THRESHOLD_PER_PAGE = 100 

def classify_and_copy_pdf(pdf_path: Path, text_dir: Path, image_dir: Path, error_dir: Path):
    """
    PDF를 분석하여 텍스트/이미지 기반으로 분류하고, 원본 PDF 파일을 해당 폴더로 복사합니다.
    """
    pdf_document = None
    try:
        pdf_document = fitz.open(pdf_path)
        
        total_chars = 0
        for page in pdf_document:
            total_chars += len(page.get_text("text"))
        
        classification_threshold = len(pdf_document) * TEXT_THRESHOLD_PER_PAGE
        
        if total_chars > classification_threshold:
            target_dir = text_dir
        else:
            target_dir = image_dir
            
        # ★★★ 수정된 부분 ★★★
        # 마크다운을 저장하는 대신, 원본 PDF 파일을 target_dir로 복사합니다.
        shutil.copy(pdf_path, target_dir / pdf_path.name)

    except Exception as e:
        # 오류 발생 시, 원본 PDF를 error 폴더로 복사
        shutil.copy(pdf_path, error_dir / pdf_path.name)
        # 오류 로그도 함께 기록
        error_log_path = error_dir / f"ERROR_{pdf_path.name}.txt"
        with open(error_log_path, "w", encoding="utf-8") as f:
            f.write(f"'{pdf_path.name}' 파일 처리 중 오류 발생:\n{e}")
        
    finally:
        if pdf_document:
            pdf_document.close()

# --- 실행 부분 ---
if __name__ == "__main__":
    pdf_root = Path("C:/skn13/final/DB2")
    output_directory = pdf_root / "_pdf_classification"
    text_based_dir = output_directory / "text_based"
    image_based_dir = output_directory / "image_based"
    error_files_dir = output_directory / "error_files"

    for dir_path in [output_directory, text_based_dir, image_based_dir, error_files_dir]:
        dir_path.mkdir(exist_ok=True)
        # 기존 파일들 정리 (선택사항)
        for item in dir_path.iterdir():
            item.unlink() if item.is_file() else shutil.rmtree(item)

    pdf_files = [p for p in pdf_root.rglob("*.pdf") if "_pdf_classification" not in str(p)]

    if not pdf_files:
        print(f"오류: '{pdf_root}' 폴더에 분류할 PDF 파일이 없습니다.")
    else:
        print(f"총 {len(pdf_files)}개의 PDF 파일을 대상으로 분류를 시작합니다.")
        for pdf_file in tqdm(pdf_files, desc="PDF 분류 진행 중"):
            classify_and_copy_pdf(pdf_file, text_based_dir, image_based_dir, error_files_dir)
        print(f"\n✅ 모든 분류 작업이 완료되었습니다. 결과를 '{output_directory}' 폴더에서 확인하세요.")

총 251개의 PDF 파일을 대상으로 분류를 시작합니다.


PDF 분류 진행 중:  46%|████▌     | 115/251 [00:16<00:04, 27.77it/s]

MuPDF error: syntax error: invalid key in dict

MuPDF error: syntax error: invalid key in dict



PDF 분류 진행 중: 100%|██████████| 251/251 [00:21<00:00, 11.72it/s]


✅ 모든 분류 작업이 완료되었습니다. 결과를 'C:\skn13\final\DB2\_pdf_classification' 폴더에서 확인하세요.


In [5]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install easyocr

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install PyMuPDF pytesseract pandas pillow

Note: you may need to restart the kernel to use updated packages.


In [5]:
import fitz
import camelot
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import pandas as pd
import logging
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
import base64
import os
from io import BytesIO

# .env 파일에서 API 키를 불러옵니다.
load_dotenv()
llm = ChatOpenAI(model_name="gpt-4o")

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def extract_text_from_page(page):
    """페이지에서 머리글/바닥글 제외 및 정렬 옵션으로 텍스트 추출"""
    try:
        page_rect = page.rect
        margin_y = page_rect.height * 0.10
        content_rect = fitz.Rect(
            page_rect.x0, page_rect.y0 + margin_y,
            page_rect.x1, page_rect.y1 - margin_y
        )
        text = page.get_text("text", clip=content_rect, sort=True).strip()
        return text if text else ""
    except Exception as e:
        logger.warning(f"텍스트 추출 중 오류 발생: {e}. 전체 페이지 텍스트를 추출합니다.")
        text = page.get_text("text", sort=True).strip()
        return text if text else ""

def correct_text_with_llm(text_to_correct: str):
    """AI를 이용해 추출된 텍스트의 오류를 수정하고 문맥을 복원합니다."""
    if not text_to_correct.strip():
        return ""

    prompt = f"""당신은 PDF에서 추출된 한국어 텍스트의 오류를 수정하는 AI 전문가입니다.
추출 과정에서 단어가 깨지거나 글자가 생략되는 오류가 발생할 수 있습니다.
아래 텍스트를 읽고, 문맥에 맞게 자연스러운 문장으로 복원해주세요.

**작업 지침:**
1. 깨진 단어나 한 글자씩 나열된 단어들을 자연스러운 단어로 합쳐주세요. (예: '목 회 잠 화회' -> '목동 방송회관 및 잠실 광고문화회관')
2. 생략된 글자나 오탈자를 문맥에 맞게 수정해주세요. (예: '공사비용구' -> '공사 비용구조')
3. 원본의 의미를 절대 변경하거나, 새로운 정보를 추가하거나, 내용을 요약해서는 안 됩니다.
4. 오직 텍스트를 교정하고 복원하는 역할만 수행해야 합니다.

**아래는 교정이 필요한 텍스트입니다:**
---
{text_to_correct}
---

**위 텍스트를 교정한 최종 결과만 응답해주세요.**"""

    try:
        messages = [HumanMessage(content=prompt)]
        corrected_text = llm.invoke(messages).content.strip()
        return corrected_text
    except Exception as e:
        logger.error(f"AI 텍스트 교정 실패: {e}")
        return text_to_correct # 실패 시 원본 텍스트 반환

def extract_tables_from_page(pdf_path: Path, page, page_num: int, tables_dir: Path, page_image: Image.Image):
    """표 추출 및 AI로 자연스러운 줄글 설명 생성"""
    md_chunks = []
    try:
        tables = camelot.read_pdf(str(pdf_path), pages=str(page_num + 1), flavor='lattice')
        if tables.n == 0: return md_chunks

        for i, table in enumerate(tables):
            csv_text = table.df.to_csv(index=False)
            prompt_text = """당신은 표에 있는 데이터를 자연스러운 문장(줄글)으로 풀어 설명하는 AI 어시스턴트입니다.
아래에는 표가 포함된 **페이지 전체 이미지**와, 그 표에서 추출한 **데이터(CSV)**가 있습니다.

**당신의 작업 지침:**
1.  **제목 찾기 및 명시:** 먼저, **페이지 전체 이미지**를 보고 표의 바로 위에 위치한 제목을 찾아주세요. 찾은 제목을 명시하고, 표에 명시된 단위(예: '단위: 억 원')가 있다면 반드시 함께 언급하며 설명을 시작해야 합니다. 만약 명확한 제목이 없다면 '제목 없음'으로 시작하세요.
2.  **자연스러운 서술:** 각 행의 데이터를 완전한 문장으로 설명해주세요. 기계적인 나열 대신, 문맥을 가진 줄글로 만들어야 합니다.
3.  **정보의 완전성:** 표에 있는 모든 행과 데이터 포인트를 반드시 언급해야 합니다.
4.  **엄격한 제한사항:** 절대 원본에 없는 내용을 분석, 요약, 해석, 추론하지 마세요.

위 지침을 엄격히 따라서 자연스러운 한국어 줄글로 결과를 생성해주세요."""

            buffer = BytesIO()
            page_image.save(buffer, format="PNG")
            img_bytes = buffer.getvalue()
            img_b64 = base64.b64encode(img_bytes).decode("utf-8")
            content_blocks = [{"type": "text", "text": prompt_text}, {"type": "text", "text": f"### 표 데이터 (CSV):\n{csv_text}"}, {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_b64}"}}]
            messages = [HumanMessage(content=content_blocks)]
            caption = llm.invoke(messages).content.strip()
            md_chunks.append(f"표 내용: {caption}")
    except Exception as e:
        logger.error(f"표 추출 실패 {pdf_path.name} p{page_num + 1}: {e}")
    return md_chunks

def extract_images_from_page(doc, page, page_num: int, pdf_stem: str, images_dir: Path):
    """이미지 추출 및 AI로 자연스러운 줄글 묘사"""
    md_chunks = []
    images = page.get_images(full=True)
    if not images: return md_chunks

    for idx, img in enumerate(images):
        try:
            xref = img[0]
            base = doc.extract_image(xref)
            img_bytes = base["image"]
            if len(img_bytes) < 3000: continue
            ext = base["ext"]
            mime = f"image/{'jpeg' if ext == 'jpg' else ext}"
            b64_img = base64.b64encode(img_bytes).decode("utf-8")
            prompt_text = """당신은 시각 자료를 객관적인 문장(줄글)으로 상세하게 묘사하는 AI 어시스턴트입니다. 당신의 임무는 아래 이미지의 모든 시각적 정보를 빠짐없이, 사람이 설명해주듯이 서술하는 것입니다. 이 결과는 벡터 데이터베이스에 사용되므로, 당신의 주관적인 해석이나 평가는 절대 포함되어서는 안 됩니다.

**당신의 작업 지침:**
1.  **이미지 종류 판단:** 이미지가 의미 있는 정보(그래프, 차트, 사진 등)인지, 의미 없는 장식용(로고, 배경 등)인지 먼저 판단해주세요. 의미 없는 이미지라면, 다른 설명 없이 오직 '(배경 이미지)' 라고만 응답해주세요.
2.  **자연스러운 묘사 (의미 있는 이미지일 경우):**
    * **그래프/차트의 경우, 다음 정보를 포함하여 줄글로 설명해주세요:** 그래프의 제목과 종류, 각 축의 정보와 단위, 각 데이터 포인트, 범례(Legend) 등
    * **일반 사진/다이어그램:** 보이는 그대로의 상황, 객체, 인물, 배경, 그리고 이미지 내 텍스트를 자연스러운 문장으로 설명해주세요.
3.  **엄격한 제한사항:** 절대 이미지의 의미를 해석하거나, 경향을 분석하거나, 결론을 도출하지 마세요.

위 지침을 엄격히 따라서 자연스러운 한국어 줄글로 결과를 생성해주세요."""
            messages = [HumanMessage(content=[{"type": "text", "text": prompt_text}, {"type": "image_url", "image_url": {"url": f"data:{mime};base64,{b64_img}"}}])]
            caption = llm.invoke(messages).content.strip()
            if caption != "(배경 이미지)":
                md_chunks.append(f"이미지 내용: {caption}")
        except Exception as e:
            logger.error(f"이미지 처리 실패 {pdf_stem} p{page_num + 1} img{idx + 1}: {e}")
    return md_chunks

def save_markdown_file(markdown_chunks, pdf_path: Path, output_dir: Path):
    """마크다운 파일 저장"""
    safe_name = "".join(c for c in pdf_path.stem if c.isalnum() or c in (' ', '_')).rstrip()
    md_path = output_dir / f"{safe_name}.md"
    with open(md_path, "w", encoding="utf-8") as f:
        f.write("\n\n".join(filter(None, markdown_chunks)))

def process_pdf_to_markdown(pdf_path: Path, output_dir: Path, images_dir: Path, tables_dir: Path):
    """PDF를 마크다운으로 변환 (AI 텍스트 교정 포함)"""
    try:
        doc = fitz.open(pdf_path)
        md_chunks = []
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            
            # 1. 텍스트 1차 추출
            raw_text = extract_text_from_page(page)
            
            # 2. AI를 통해 추출된 텍스트 교정
            corrected_text = correct_text_with_llm(raw_text)
            if corrected_text:
                md_chunks.append(corrected_text)
            
            # 3. 표와 이미지 처리는 이전과 동일하게 진행
            pix = page.get_pixmap(dpi=200)
            page_image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            md_chunks.extend(extract_tables_from_page(pdf_path, page, page_num, tables_dir, page_image))
            md_chunks.extend(extract_images_from_page(doc, page, page_num, pdf_path.stem, images_dir))
        
        save_markdown_file(md_chunks, pdf_path, output_dir)
        doc.close()
    except Exception as e:
        logger.error(f"PDF 처리 실패 {pdf_path.name}: {e}")
        raise

def main():
    """메인 실행 함수"""
    pdf_root = Path(r"C:\skn13\final\DB2\내부문서\재무성과")
    output_dir = pdf_root / "_markdown_output"
    images_dir = output_dir / "_images"
    tables_dir = output_dir / "_tables"
    
    output_dir.mkdir(parents=True, exist_ok=True)
    images_dir.mkdir(parents=True, exist_ok=True)
    tables_dir.mkdir(parents=True, exist_ok=True)
    
    pdf_files = [p for p in pdf_root.rglob("*.pdf") if "_markdown_output" not in str(p)]
    if not pdf_files:
        logger.warning(f"PDF 파일을 찾을 수 없습니다: {pdf_root}")
        return
    
    logger.info(f"총 {len(pdf_files)}개의 PDF 파일을 처리합니다.")
    
    # 전체 파일 처리 시 '[:3]' 부분을 지우고 'files_to_process = pdf_files'로 변경
    files_to_process = pdf_files[:3] 
    
    success_count = 0
    for pdf_file in tqdm(files_to_process, desc="PDF 변환 중"):
        try:
            process_pdf_to_markdown(pdf_file, output_dir, images_dir, tables_dir)
            success_count += 1
        except Exception:
            pass
    
    logger.info(f"처리 완료: {success_count}/{len(files_to_process)}개 성공")
    logger.info(f"결과 위치: {output_dir}")

if __name__ == "__main__":
    main()

2025-08-06 15:46:52,968 - INFO - 총 53개의 PDF 파일을 처리합니다.
PDF 변환 중:   0%|          | 0/3 [00:00<?, ?it/s]2025-08-06 15:46:54,311 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 15:47:00,767 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 15:47:02,304 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 15:47:03,303 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 15:47:08,863 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 15:47:21,867 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 15:47:28,303 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 15:47:53,809 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 20

In [6]:
import fitz
import camelot
from pathlib import Path
from tqdm import tqdm
from PIL import Image
import pandas as pd
import logging
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
import base64
import os
from io import BytesIO

# .env 파일에서 API 키를 불러옵니다.
load_dotenv()
llm = ChatOpenAI(model_name="gpt-4o")

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def extract_text_from_page(page):
    """페이지에서 머리글/바닥글 제외 및 정렬 옵션으로 텍스트 추출"""
    try:
        page_rect = page.rect
        margin_y = page_rect.height * 0.10
        content_rect = fitz.Rect(
            page_rect.x0, page_rect.y0 + margin_y,
            page_rect.x1, page_rect.y1 - margin_y
        )
        text = page.get_text("text", clip=content_rect, sort=True).strip()
        return text if text else ""
    except Exception as e:
        logger.warning(f"텍스트 추출 중 오류 발생: {e}. 전체 페이지 텍스트를 추출합니다.")
        text = page.get_text("text", sort=True).strip()
        return text if text else ""

def correct_text_with_llm(text_to_correct: str):
    """AI를 이용해 추출된 텍스트의 오류를 수정하고 문맥을 복원합니다."""
    if not text_to_correct.strip():
        return ""

    prompt = f"""당신은 PDF에서 추출된 한국어 텍스트의 오류를 수정하는 AI 전문가입니다.
추출 과정에서 단어가 깨지거나 글자가 생략되는 오류가 발생할 수 있습니다.
아래 텍스트를 읽고, 문맥에 맞게 자연스러운 문장으로 복원해주세요.

**작업 지침:**
1. 깨진 단어나 한 글자씩 나열된 단어들을 자연스러운 단어로 합쳐주세요. (예: '목 회 잠 화회' -> '목동 방송회관 및 잠실 광고문화회관')
2. 생략된 글자나 오탈자를 문맥에 맞게 수정해주세요. (예: '공사비용구' -> '공사 비용구조')
3. 원본의 의미를 절대 변경하거나, 새로운 정보를 추가하거나, 내용을 요약해서는 안 됩니다.
4. 오직 텍스트를 교정하고 복원하는 역할만 수행해야 합니다.

**아래는 교정이 필요한 텍스트입니다:**
---
{text_to_correct}
---

**위 텍스트를 교정한 최종 결과만 응답해주세요.**"""

    try:
        messages = [HumanMessage(content=prompt)]
        corrected_text = llm.invoke(messages).content.strip()
        return corrected_text
    except Exception as e:
        logger.error(f"AI 텍스트 교정 실패: {e}")
        return text_to_correct # 실패 시 원본 텍스트 반환

def extract_tables_from_page(pdf_path: Path, page, page_num: int, tables_dir: Path, page_image: Image.Image):
    """표 추출 및 AI로 자연스러운 줄글 설명 생성"""
    md_chunks = []
    try:
        tables = camelot.read_pdf(str(pdf_path), pages=str(page_num + 1), flavor='lattice')
        if tables.n == 0: return md_chunks

        for i, table in enumerate(tables):
            csv_text = table.df.to_csv(index=False)
            prompt_text = """당신은 표에 있는 데이터를 자연스러운 문장(줄글)으로 풀어 설명하는 AI 어시스턴트입니다.
아래에는 표가 포함된 **페이지 전체 이미지**와, 그 표에서 추출한 **데이터(CSV)**가 있습니다.

**당신의 작업 지침:**
1.  **제목 찾기 및 명시:** 먼저, **페이지 전체 이미지**를 보고 표의 바로 위에 위치한 제목을 찾아주세요. 찾은 제목을 명시하고, 표에 명시된 단위(예: '단위: 억 원')가 있다면 반드시 함께 언급하며 설명을 시작해야 합니다. 만약 명확한 제목이 없다면 '제목 없음'으로 시작하세요.
2.  **자연스러운 서술:** 각 행의 데이터를 완전한 문장으로 설명해주세요. 기계적인 나열 대신, 문맥을 가진 줄글로 만들어야 합니다.
3.  **정보의 완전성:** 표에 있는 모든 행과 데이터 포인트를 반드시 언급해야 합니다.
4.  **엄격한 제한사항:** 절대 원본에 없는 내용을 분석, 요약, 해석, 추론하지 마세요.

위 지침을 엄격히 따라서 자연스러운 한국어 줄글로 결과를 생성해주세요."""

            buffer = BytesIO()
            page_image.save(buffer, format="PNG")
            img_bytes = buffer.getvalue()
            img_b64 = base64.b64encode(img_bytes).decode("utf-8")
            content_blocks = [{"type": "text", "text": prompt_text}, {"type": "text", "text": f"### 표 데이터 (CSV):\n{csv_text}"}, {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{img_b64}"}}]
            messages = [HumanMessage(content=content_blocks)]
            caption = llm.invoke(messages).content.strip()
            md_chunks.append(f"표 내용: {caption}")
    except Exception as e:
        logger.error(f"표 추출 실패 {pdf_path.name} p{page_num + 1}: {e}")
    return md_chunks

def extract_images_from_page(doc, page, page_num: int, pdf_stem: str, images_dir: Path):
    """이미지 추출 및 AI로 자연스러운 줄글 묘사"""
    md_chunks = []
    images = page.get_images(full=True)
    if not images: return md_chunks

    for idx, img in enumerate(images):
        try:
            xref = img[0]
            base = doc.extract_image(xref)
            img_bytes = base["image"]
            if len(img_bytes) < 3000: continue
            ext = base["ext"]
            mime = f"image/{'jpeg' if ext == 'jpg' else ext}"
            b64_img = base64.b64encode(img_bytes).decode("utf-8")
            prompt_text = """당신은 시각 자료를 객관적인 문장(줄글)으로 상세하게 묘사하는 AI 어시스턴트입니다. 당신의 임무는 아래 이미지의 모든 시각적 정보를 빠짐없이, 사람이 설명해주듯이 서술하는 것입니다. 이 결과는 벡터 데이터베이스에 사용되므로, 당신의 주관적인 해석이나 평가는 절대 포함되어서는 안 됩니다.

**당신의 작업 지침:**
1.  **이미지 종류 판단:** 이미지가 의미 있는 정보(그래프, 차트, 사진 등)인지, 의미 없는 장식용(로고, 배경 등)인지 먼저 판단해주세요. 의미 없는 이미지라면, 다른 설명 없이 오직 '(배경 이미지)' 라고만 응답해주세요.
2.  **자연스러운 묘사 (의미 있는 이미지일 경우):**
    * **그래프/차트의 경우, 다음 정보를 포함하여 줄글로 설명해주세요:** 그래프의 제목과 종류, 각 축의 정보와 단위, 각 데이터 포인트, 범례(Legend) 등
    * **일반 사진/다이어그램:** 보이는 그대로의 상황, 객체, 인물, 배경, 그리고 이미지 내 텍스트를 자연스러운 문장으로 설명해주세요.
3.  **엄격한 제한사항:** 절대 이미지의 의미를 해석하거나, 경향을 분석하거나, 결론을 도출하지 마세요.

위 지침을 엄격히 따라서 자연스러운 한국어 줄글로 결과를 생성해주세요."""
            messages = [HumanMessage(content=[{"type": "text", "text": prompt_text}, {"type": "image_url", "image_url": {"url": f"data:{mime};base64,{b64_img}"}}])]
            caption = llm.invoke(messages).content.strip()
            if caption != "(배경 이미지)":
                md_chunks.append(f"이미지 내용: {caption}")
        except Exception as e:
            logger.error(f"이미지 처리 실패 {pdf_stem} p{page_num + 1} img{idx + 1}: {e}")
    return md_chunks

def save_markdown_file(markdown_chunks, pdf_path: Path, output_dir: Path):
    """마크다운 파일 저장"""
    safe_name = "".join(c for c in pdf_path.stem if c.isalnum() or c in (' ', '_')).rstrip()
    md_path = output_dir / f"{safe_name}.md"
    with open(md_path, "w", encoding="utf-8") as f:
        f.write("\n\n".join(filter(None, markdown_chunks)))

def process_pdf_to_markdown(pdf_path: Path, output_dir: Path, images_dir: Path, tables_dir: Path):
    """PDF를 마크다운으로 변환 (AI 텍스트 교정 포함)"""
    try:
        doc = fitz.open(pdf_path)
        md_chunks = []
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            
            raw_text = extract_text_from_page(page)
            
            corrected_text = correct_text_with_llm(raw_text)
            if corrected_text:
                md_chunks.append(corrected_text)
            
            pix = page.get_pixmap(dpi=200)
            page_image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            
            md_chunks.extend(extract_tables_from_page(pdf_path, page, page_num, tables_dir, page_image))
            md_chunks.extend(extract_images_from_page(doc, page, page_num, pdf_path.stem, images_dir))
        
        save_markdown_file(md_chunks, pdf_path, output_dir)
        doc.close()
    except Exception as e:
        logger.error(f"PDF 처리 실패 {pdf_path.name}: {e}")
        raise

def main():
    """메인 실행 함수"""
    pdf_root = Path(r"C:\skn13\final\DB2\내부문서\test")
    output_dir = pdf_root / "_markdown_output"
    images_dir = output_dir / "_images"
    tables_dir = output_dir / "_tables"
    
    output_dir.mkdir(parents=True, exist_ok=True)
    images_dir.mkdir(parents=True, exist_ok=True)
    tables_dir.mkdir(parents=True, exist_ok=True)
    
    pdf_files = [p for p in pdf_root.rglob("*.pdf") if "_markdown_output" not in str(p)]
    if not pdf_files:
        logger.warning(f"PDF 파일을 찾을 수 없습니다: {pdf_root}")
        return
    
    logger.info(f"총 {len(pdf_files)}개의 PDF 파일을 처리합니다.")
    
    # ★★★ 전체 파일을 대상으로 처리하도록 수정 ★★★
    files_to_process = pdf_files 
    
    success_count = 0
    for pdf_file in tqdm(files_to_process, desc="PDF 변환 중"):
        try:
            process_pdf_to_markdown(pdf_file, output_dir, images_dir, tables_dir)
            success_count += 1
        except Exception:
            pass
    
    logger.info(f"처리 완료: {success_count}/{len(files_to_process)}개 성공")
    logger.info(f"결과 위치: {output_dir}")

if __name__ == "__main__":
    main()

2025-08-06 16:20:55,656 - INFO - 총 9개의 PDF 파일을 처리합니다.
PDF 변환 중:   0%|          | 0/9 [00:00<?, ?it/s]2025-08-06 16:20:57,012 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 16:20:58,431 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 16:20:59,473 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 16:21:04,990 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 16:21:15,300 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 16:21:23,012 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 16:21:38,184 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-08-06 16:21:47,514 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200